In [17]:
# !pip install import_ipynb
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [18]:
import import_ipynb

In [21]:
# 지난번 데이터전처리 완료한 독립변수 반환
from Entire_process_Housing_model import housing, full_pipeline

ModuleNotFoundError: No module named 'Entire_process_Housing_model'

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()

In [ ]:
# 종속변수(레이블)도 소환
from Entire_process_Housing_model import housing_labels
housing_labels

#### 세부튜닝 - 그리드 서치 cv

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators':[3,10,30], 'max_features':[2,4,6,8]}, 
              {'bootstrap':[False], 'n_estimators':[3, 10], 'max_features':[2,3,4]},
             ]

In [ ]:
# 그리드 서치
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
# 최적의 그리드 서치값 도출
grid_search.best_params_

In [ ]:
grid_search.best_estimator_ # 최적의 검출 패러미터

In [ ]:
# 평가점수 패러미터별 확인
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(f"RMSE점수 : {np.sqrt(-mean_score)}, {params}")

In [ ]:
# 서포트 벡터머신을 그리드 서치를 통해 점수 계산

# SVM 회귀 모델 사용
from sklearn import svm

grid_search_svm = GridSearchCV(estimator=svm.SVR(),
             param_grid={'C': [1, 2, 5, 7, 10], 'gamma': [0.001, 0.002, 0.01, 0.02, 0.1, 0.2, 0.5],
            'kernel': ('linear', 'rbf')}, cv = 5, 
            scoring='neg_mean_squared_error', return_train_score=True)
grid_search_svm

In [ ]:
grid_search_svm.fit(housing_prepared, housing_labels)

In [ ]:
# SVM의 최적 parameter 도출
grid_search_svm.best_params_

In [ ]:
grid_search_svm.best_estimator_ # 최적의 검출 패러미터

In [ ]:
# 평가점수 (svm) 패러미터별 확인
cvres_svm = grid_search_svm.cv_results_
for mean_score, params in zip(cvres_svm["mean_test_score"], cvres_svm["params"]):
    print(f"RMSE점수 : {np.sqrt(-mean_score)}, {params}")

#### 모델 지정, 피쳐 중요도 분석

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances
# 피쳐의 상대적인 중요도

In [ ]:
# 피쳐 이름 서술
from Entire_process_Housing_model import num_attribs
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs+extra_attribs+cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

#### 최종모델 평가

In [ ]:
from Entire_process_Housing_model import strat_test_set
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()
# 테스트셋

In [ ]:
from sklearn.metrics import mean_squared_error
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

In [ ]:
# 신뢰구간 계산
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions-y_test) **2
np.sqrt(stats.t.interval(confidence, len(squared_errors) -1, 
                        loc=squared_erros.mean(),
                        scale=stats.sem(squared_errors)))